In [11]:
from encoder import Encoder
import torch
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

In [12]:
W2V_PATH = "/home/jingjing/Desktop/InferSent-master/dataset/GloVe/glove.840B.300d.txt"

In [13]:
f = Encoder()
sentences = ['The Moon is filled wit craters.', 'It has no light of its own.', 'It gets its light from the Sun.']

data = [['Memories of childhood are unforgettable.', 'I was four years old when my grandfather died.',
             'I clearly remember how everybody in the house was weeping.'], ['Today is sunny', 'We should go out for a picnic.', 'Love the weather.']]

f.zero_grad()

f.set_w2v_path(W2V_PATH)

In [14]:
def make_target(context_size, dim):
    targets = np.zeros((dim, dim))
    ctxt_sent_pos = list(range(-context_size, context_size+1))
    ctxt_sent_pos.remove(0)
    for ctxt in ctxt_sent_pos:
        targets += np.eye(3, k=ctxt)
    targets_sum = np.sum(targets,axis=1, keepdims=True)
    targets = targets / targets_sum
    targets = torch.from_numpy(targets)
    return targets

In [15]:
targets = make_target(1, 3)

In [16]:
def loss_fn(pred, target):
    m = nn.Softmax(dim=-1)
    s_pred = m(pred)
    losses = F.binary_cross_entropy_with_logits(s_pred, target)
    return losses

In [17]:
with torch.no_grad():
    f.build_vocab(sentences, True)
    embeddings = f.encode(sentences, 3)
    loss = loss_fn(embeddings, targets.float())
    print("loss before training: ", loss)

Found 20(/20) words with w2v vectors
Vocab size : 20
loss before training:  tensor(0.7914)


In [18]:
optimizer = optim.Adam(f.parameters(), lr=0.0005)

In [19]:
for epoch in range(20):
    for instance in data:
        optimizer.zero_grad()
        if epoch==0:
            f.build_vocab(instance, True)
        scores = f.encode(instance, 3)
        loss = loss_fn(scores, targets.float())
        print(loss)
        loss.backward()
        optimizer.step()

Found 25(/25) words with w2v vectors
Vocab size : 25
tensor(0.8036)
Found 16(/16) words with w2v vectors
Vocab size : 16
tensor(0.8032)
tensor(0.7549)
tensor(0.6783)
tensor(0.7456)
tensor(0.6776)
tensor(0.7420)
tensor(0.6776)
tensor(0.7416)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)
tensor(0.7415)
tensor(0.6776)


In [20]:
with torch.no_grad():
    f.build_vocab(sentences, True)
    embeddings = f.encode(sentences, 3)
    loss = loss_fn(embeddings, targets.float())
    print("loss after training: ", loss)

Found 20(/20) words with w2v vectors
Vocab size : 20
loss after training:  tensor(0.6777)
